In [1]:
%matplotlib inline

In [3]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt

In [4]:
iris = datasets.load_iris()
type(iris)

sklearn.utils.Bunch

In [5]:
X = iris["data"]
type(X)

numpy.ndarray

In [6]:
X.shape

(150, 4)

In [7]:
X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

_Iris dataset_ has **too few** instances. Let's take _Fashion MNIST_ instead.

In [8]:
import tensorflow as tf

In [14]:
a = tf.constant([0,1,2], dtype=tf.float32)
tf.math.reduce_sum(a)

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [67]:
import tensorflow.keras as keras
K = keras.backend
#K.math.reduce_sum(a)

```python
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-16-9c036f0ad0a4> in <module>
      1 import tensorflow.keras as keras
      2 K = keras.backend
----> 3 K.math.reduce_sum(a)

AttributeError: module 'tensorflow.keras.backend' has no attribute 'math'
```

#### Instead, just do `K.sum(a)`

In [17]:
K.sum(a)

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [43]:
def wicked_sigmoid(y_true, y_pred):
    """
    Our customized loss function
    Example: (4-class classification)
        y_pred = [ -500.0, -37.92, 99.99]
        y_true = [0, 0, 0]  # meaning that the true class is the fourth one.
    """
    s = tf.math.reduce_sum(y_pred)
    print(f"s = {s}")
    last_proba = 1 - tf.math.sigmoid(s)
    print(f"tf.math.sigmoid(s) = {tf.math.sigmoid(s)}")
    print(f"last_proba = 1 - tf.math.sigmoid(s) = {last_proba}")
    first_probas = tf.math.sigmoid(s) * tf.nn.softmax(y_pred)
    print(f"tf.nn.softmax(y_pred) = {tf.nn.softmax(y_pred)}")
    print(f"first_probas = {first_probas}")
    yy_pred = tf.concat([first_probas, [last_proba]], axis=0)
    print(f"yy_pred = {yy_pred}")
    yy_true = tf.cond(tf.math.reduce_sum(y_true) < 0.01,
                      true_fn=lambda: tf.concat([y_true, [1]], 0),
                      false_fn=lambda: tf.concat([y_true, [0]], 0),
    )
    print(f"yy_true = {yy_true}")
    return tf.nn.softmax_cross_entropy_with_logits(
        labels=yy_true,
        logits=yy_pred,
    )

In [44]:
wicked_sigmoid(tf.constant([0,0,0], dtype=tf.float32), tf.constant([-500.0, -37.92, 99.99]))
#crooked_sigmoid(tf.constant([0,0,0], dtype=tf.float32), tf.constant([-500.0, -37.92, 99.99]))

s = -437.92999267578125
tf.math.sigmoid(s) = 0.0
last_proba = 1 - tf.math.sigmoid(s) = 1.0
tf.nn.softmax(y_pred) = [0. 0. 1.]
first_probas = [0. 0. 0.]
yy_pred = [0. 0. 0. 1.]
yy_true = [0. 0. 0. 1.]


<tf.Tensor: shape=(), dtype=float32, numpy=0.74366844>

### But...
We are not so sure about the shape of `y_true` and `y_pred` in keras. How can we make sure of that?

In [45]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_labels.shape

4423680/4422102 [==============================] - 9s 2us/step


(60000,)

In [46]:
train_labels[0]

9

In [47]:
tf.one_hot(9, depth=10)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)>

In [48]:
tf.one_hot(train_labels, depth=10)

<tf.Tensor: shape=(60000, 10), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [49]:
tf.one_hot(train_labels, depth=10)[:,:-1]

<tf.Tensor: shape=(60000, 9), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [50]:
tf.one_hot(train_labels, depth=10)[:,:-1].shape

TensorShape([60000, 9])

In [51]:
type(train_images)

numpy.ndarray

In [52]:
from sklearn.preprocessing import OneHotEncoder

In [63]:
encoder = OneHotEncoder()
#hey = encoder.fit_transform(train_labels[np.newaxis])
hey = encoder.fit_transform(train_labels.reshape((-1,1)))
hey

<60000x10 sparse matrix of type '<class 'numpy.float64'>'
	with 60000 stored elements in Compressed Sparse Row format>

In [57]:
train_images = train_images / 255.0
test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=wicked_sigmoid,
              metrics=['accuracy'])

In [58]:
model.fit(train_images, tf.one_hot(train_labels, depth=10)[:,:-1], epochs=10)

Epoch 1/10
s = Tensor("wicked_sigmoid/Sum:0", shape=(), dtype=float32)
tf.math.sigmoid(s) = Tensor("wicked_sigmoid/Sigmoid_1:0", shape=(), dtype=float32)
last_proba = 1 - tf.math.sigmoid(s) = Tensor("wicked_sigmoid/sub:0", shape=(), dtype=float32)
tf.nn.softmax(y_pred) = Tensor("wicked_sigmoid/Softmax_1:0", shape=(32, 10), dtype=float32)
first_probas = Tensor("wicked_sigmoid/mul:0", shape=(32, 10), dtype=float32)


ValueError: in user code:

    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-43-ec2c57535c9c>:16 wicked_sigmoid  *
        yy_pred = tf.concat([first_probas, [last_proba]], axis=0)
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1654 concat
        return gen_array_ops.concat_v2(values=values, axis=axis, name=name)
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/ops/gen_array_ops.py:1221 concat_v2
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/framework/op_def_library.p

In [64]:
hey.toarray().shape

(60000, 10)

In [65]:
hey.toarray()

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
model.fit(train_images, hey.toarray()[:,:-1], epochs=10)

Epoch 1/10
s = Tensor("wicked_sigmoid/Sum:0", shape=(), dtype=float32)
tf.math.sigmoid(s) = Tensor("wicked_sigmoid/Sigmoid_1:0", shape=(), dtype=float32)
last_proba = 1 - tf.math.sigmoid(s) = Tensor("wicked_sigmoid/sub:0", shape=(), dtype=float32)
tf.nn.softmax(y_pred) = Tensor("wicked_sigmoid/Softmax_1:0", shape=(32, 10), dtype=float32)
first_probas = Tensor("wicked_sigmoid/mul:0", shape=(32, 10), dtype=float32)


ValueError: in user code:

    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-43-ec2c57535c9c>:16 wicked_sigmoid  *
        yy_pred = tf.concat([first_probas, [last_proba]], axis=0)
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1654 concat
        return gen_array_ops.concat_v2(values=values, axis=axis, name=name)
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/ops/gen_array_ops.py:1221 concat_v2
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /home/phunc20/.virtualenvs/tf2.3.0-py3.8/lib/python3.8/site-packages/tensorflow/python/framework/op_def_library.p